In [ ]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import cv2
from IPython.display import display
import math, random
import time, datetime, sys, os, shutil
import operator

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#tf.config.optimizer.set_jit(True)
#tf.debugging.set_log_device_placement(True)
print(tf.config.experimental.list_physical_devices())

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
VALIDATION_STEPS = 30


%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons(SELECTED_TAXONS)
id_map_inv = get_selected_taxons(SELECTED_TAXONS, inv=True)
n_classes = len(list(id_map.keys()))

tf.test.is_gpu_available()

# Handle data

## Data generator

In [ ]:
# Sampling the dataset to fit the ImageDataGenerator
files, _, _ = get_dataset()
sample_files = np.random.permutation(files)[0:int(0.4*len(files))]
new_size = 50
#fit_dataset = np.array([cv2.resize(cv2.imread(file), dsize=(new_size, new_size), interpolation=cv2.INTER_CUBIC) for file in sample_files])
fit_dataset = np.array([cv2.imread(file) for file in sample_files]).astype("float32")
del files, sample_files
print(fit_dataset.shape)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255.,
                         rotation_range=90, 
                         brightness_range=[0.8,1.2], 
                         horizontal_flip=True, 
                         vertical_flip=True,
                         fill_mode='nearest',
                         width_shift_range=40,
                         height_shift_range=40,
                         zoom_range=0.2,
                         validation_split=0.2,
                         #featurewise_std_normalization=True,
                         data_format="channels_last") 
#datagen.fit(fit_dataset)
del fit_dataset

## Setting up the flows

In [ ]:
classes_array = get_id_array(id_map)
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=True,
    classes=classes_array,
    batch_size=BATCH_SIZE,
    subset="training",
    seed=27)
val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    class_mode="categorical",
    color_mode="rgb",
    shuffle=True,
    classes=classes_array,
    batch_size=BATCH_SIZE,
    subset="validation",
    seed=27)

train_spe = train_generator.samples // BATCH_SIZE
val_spe = val_generator.samples // BATCH_SIZE

## Testing

In [ ]:
i = 0
stop = False
for batch in train_generator:
    images = batch[0]
    labels = batch[1]
    for i in range(images.shape[0]):
        print(np.argmax(labels[i]))
        image = (images[i,:,:,:]*255).astype('uint8')
        #print(image)
        display(Image.fromarray(image))
        i+=1
        if i>=1:
            stop = True
            break
    if stop: break;
    #display()

# Model desgin

In [ ]:
# fetching base model
#base_model = Xception(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling=None)
input_tensor = Input(shape=(256, 256, 3))
base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)

In [ ]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(230, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

## Setting callbacks

In [ ]:
# Setting tensorboard
delete_all_files_in_folder(LOG_DIR)
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
filename="model-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_callback = ModelCheckpoint(os.path.join(SAVED_MODELS_ROOT, filename), 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='auto', 
                             save_freq='epoch')

## New layers only

In [ ]:
n_epochs = 5
last_epoch = get_last_epoch(SAVED_MODELS_ROOT)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=True, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, checkpoint_callback],
          initial_epoch=last_epoch)

## Training 2 last inceptions blocks

In [ ]:
n_epochs = 10
last_epoch = get_last_epoch(SAVED_MODELS_ROOT)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=True, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, checkpoint_callback],
          initial_epoch=last_epoch)

## Training full model

In [ ]:
n_epochs = 40
last_epoch = get_last_epoch(SAVED_MODELS_ROOT)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=True, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, checkpoint_callback],
          initial_epoch=last_epoch)

## Saving model

In [ ]:
save_model(model, SAVED_MODELS_ROOT)